In [1]:
import pandas as pd
import os

# Define the directory where your Excel files are located
directory = 'C:\\Users\\June Lin\\Oracle Content\\June Lin\\Recurring Reports\\EMEA\\Rename_EMEA\\'


# Ensure the directory exists
if not os.path.exists(directory):
    os.makedirs(directory)

# List of Excel files in the directory
excel_files = [file for file in os.listdir(directory) if file.endswith(('.xls', '.xlsx'))]

In [2]:
# Dictionary to store assignment date range for each file
assignment_dates = {}

# Columns to convert to datetime format
date_columns = ['Contract Close Date', 'Funding Date', 'Assignment Date', 'Payment Due Date']

# Read assignment date from each Excel file
for file in excel_files:
    # Construct the file path
    file_path = os.path.join(directory, file)
    
    # Read the Excel file and get the first sheet name
    if file.endswith('.xls'):
        # Read the HTML table
        df = pd.read_html(file_path)[0]
        sheet_name = 'Payment Details'  # Default sheet name as HTML does not store sheet name
    else:
        xls = pd.ExcelFile(file_path)
        sheet_name = xls.sheet_names[0]
        df = pd.read_excel(file_path, sheet_name=sheet_name)  # Read .xlsx file

    # Convert specified columns to datetime format and format them as m/d/YYYY
    for col in date_columns:
        if col in df.columns:
            # Attempt to convert the column to datetime
            df[col] = pd.to_datetime(df[col].str.strip(), format='%d-%b-%Y', errors='coerce')  # Convert to datetime and strip any spaces
            df[col] = df[col].dt.strftime('%#m/%#d/%Y')  # Format as m/d/YYYY
            df[col] = df[col].fillna('')  # Fill NaT with empty string or a default date if required

    # Extract assignment date from column 16, row 2 (since columns and rows are 0-indexed in pandas)
    assignment_date = df.iloc[1, 15]  # Ensure the correct column index
    assignment_dates[file] = pd.to_datetime(assignment_date)

    # Construct the new file path for saving as .xlsx
    new_file_path = os.path.join(directory, os.path.splitext(file)[0] + '.xlsx')

    # Save the DataFrame as .xlsx
    with pd.ExcelWriter(new_file_path, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)


In [3]:
# Read the table listing file name and assignment date range
criteria_df = pd.read_excel(r'C:\Users\June Lin\Oracle Content\June Lin\Recurring Reports\EMEA\assignment_dateRange_EMEA.xlsx') # Update with your criteria table file path

In [4]:
# Check if assignment date falls under the criteria and rename the file accordingly
for file, date in assignment_dates.items():
    for _, row in criteria_df.iterrows():
        if row['Start Date'] <= date <= row['End Date']:
            new_file_name = f"{row['File Name']}"  # Ensure the new file name has the .xlsx extension
            new_file_path = os.path.join(directory, new_file_name)
            
            # Check if a file with the new name already exists and handle the conflict
            if os.path.exists(new_file_path):
                print(f"File {new_file_name} already exists. Skipping rename for {file}.")
                continue
            
            # Rename the file
            original_file_path = os.path.join(directory, os.path.splitext(file)[0] + '.xlsx')
            if os.path.exists(original_file_path):
                os.rename(original_file_path, new_file_path)
            else:
                print(f"Original file {original_file_path} does not exist. Skipping.")
            break

print("Files renamed successfully!")

Files renamed successfully!
